In [1]:
from symmer.symplectic import (
    PauliwordOp, QuantumState, random_PauliwordOp, random_QuantumState,
    array_to_QuantumState, find_symmetry_basis
)
from symmer.chemistry import MoleculeBuilder
from symmer.utils import lp_norm, exact_gs_energy
import numpy as np

In [2]:
geometry=[
    ('H', (0.0,0.0,0.0)),
    ('H', (2.45366053071732,0.0,0.0)),
    ('H', (2.45366053071732,2.45366053071732,0.0)),
    ('H', (0.0,2.45366053071732,0.0))
     ]    
    
basis = 'sto-3g'
multiplicity = 1
charge = 0

molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True)

Molecule geometry:
H	0.0	0.0	0.0
H	2.45366053071732	0.0	0.0
H	2.45366053071732	2.45366053071732	0.0
H	0.0	2.45366053071732	0.0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -1.3334202453959019
MP2 energy:  -1.616081960776292
CCSD energy: -1.8261563137199488
FCI energy:  -1.8743108952541838


Number of qubits: 8


In [8]:
#n_qubits = 10
#n_terms  = 100

H = molecule.H_q# random_PauliwordOp(n_qubits, n_terms, complex_coeffs=False)
#print(H)
gs_nrg, gs_vec = exact_gs_energy(H.to_sparse_matrix)
print(f'Ground state energy = {gs_nrg}')
S = find_symmetry_basis(H)
S

Ground state energy = -1.8743108952541774


 1 ZIIZIZZI 
 1 IZIZIZIZ 
 1 IIZZIIII 
 1 IIIIZZII

In [9]:
gs_psi = array_to_QuantumState(gs_vec)
gs_psi.sectors_present(S)

array([ 1.+0.j,  1.+0.j, -1.+0.j, -1.+0.j])

In [10]:
hf_psi = QuantumState([molecule.hf_array])
hf_psi.sectors_present(S)

array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])

Initialize the algorithm with a random state:

In [11]:
def gram_schmidt_step(v, orthogonal_set):
    u_out = v.copy()
    for u in orthogonal_set:
        u_norm = np.square(lp_norm(u.coeff_vector))
        uv_inner_prod = u.conjugate * v
        u_out -= u.multiply_by_constant(uv_inner_prod/u_norm)
    return u_out

In [84]:
v_init = random_QuantumState(H.n_qubits, n_terms=10, normalized=True)
v_init = v_init.cleanup().normalize
print(v_init)
print(v_init.conjugate * v_init)
w_init_prime = H * v_init
alpha_init = w_init_prime.conjugate * v_init
w_init = w_init_prime - v_init.multiply_by_constant(alpha_init)
w_init = w_init.sort()[0].normalize
print(w_init)

 0.204+0.334j |00000001> +
 0.308+0.004j |00100001> +
 0.076+0.223j |00101010> +
 0.349+0.318j |00111100> +
 0.089+0.115j |00111101> +
 0.019+0.179j |01110101> +
 0.365+0.077j |01111101> +
 0.310+0.141j |10011011> +
 0.270+0.041j |10100111> +
 0.197+0.227j |10101010>
(0.9999999999999998+0j)
 0.522+0.853j |00000001>


From the randomized starting point, construct the next state as follows:

In [85]:
2**8

256

In [99]:
w_current = w_init.copy()
v_current = v_init.copy()

w_list = [w_current]
v_list = [v_current]
for j in range(10):
    
    # construct v_next from w_current
    beta_next = lp_norm(w_current.coeff_vector)
    print(f'beta_next = {beta_next}')
    if beta_next>1e-5:
        v_next = w_current.multiply_by_constant(1/beta_next)
    else:
        v_next = random_QuantumState(n_qubits, n_terms=np.random.randint(0,2**n_qubits))
        v_next = gram_schmidt_step(v=v_next, orthogonal_set=v_list) # orthogonalize w.r.t to previous v vectors
    
    v_next = v_next.sort()[0]
    v_next.coeff_vector = np.array([1])
    v_list.append(v_next)
    
    # construct w_next from w_current and v_next
    w_next_prime = H * v_next
    alpha_next = w_next_prime.conjugate * v_next
    w_next = w_next_prime - v_next.multiply_by_constant(alpha_next) - v_current.multiply_by_constant(beta_next)
    
    w_next = w_next.sort()[0]
    w_next.coeff_vector = np.array([1])
    w_list.append(w_next)
    
    print(v_next, w_next)
    print(v_next.conjugate * H * v_next)
    # update the current v,w vectors and repeat
    w_current = w_next
    v_current = v_next

beta_next = 0.9999999999999999
 1.000 |00000001>  1.000 |00111100>
(0.16295917253578637+0j)
beta_next = 1.0
 1.000 |00111100>  1.000 |00000001>
(-1.226923982519471+0j)
beta_next = 1.0
 1.000 |00000001>  1.000 |00111100>
(0.16295917253578637+0j)
beta_next = 1.0
 1.000 |00111100>  1.000 |00000001>
(-1.226923982519471+0j)
beta_next = 1.0
 1.000 |00000001>  1.000 |00111100>
(0.16295917253578637+0j)
beta_next = 1.0
 1.000 |00111100>  1.000 |00000001>
(-1.226923982519471+0j)
beta_next = 1.0
 1.000 |00000001>  1.000 |00111100>
(0.16295917253578637+0j)
beta_next = 1.0
 1.000 |00111100>  1.000 |00000001>
(-1.226923982519471+0j)
beta_next = 1.0
 1.000 |00000001>  1.000 |00111100>
(0.16295917253578637+0j)
beta_next = 1.0
 1.000 |00111100>  1.000 |00000001>
(-1.226923982519471+0j)


In [100]:
from scipy import sparse as sp

V = sp.hstack([v.to_sparse_matrix for v in v_list])

In [101]:
T = (V.H @ H.to_sparse_matrix @ V).real

In [102]:
print(abs(T.toarray())>1e-10)

[[ True  True  True  True  True  True  True  True  True  True  True]
 [ True  True False  True False  True False  True False  True False]
 [ True False  True False  True False  True False  True False  True]
 [ True  True False  True False  True False  True False  True False]
 [ True False  True False  True False  True False  True False  True]
 [ True  True False  True False  True False  True False  True False]
 [ True False  True False  True False  True False  True False  True]
 [ True  True False  True False  True False  True False  True False]
 [ True False  True False  True False  True False  True False  True]
 [ True  True False  True False  True False  True False  True False]
 [ True False  True False  True False  True False  True False  True]]


In [103]:
T_gs_nrg, T_gs_vec = exact_gs_energy(T)
print(f'Grounds state energy of H is {gs_nrg}, of T it is {T_gs_nrg}')

Grounds state energy of H is -1.8743108952541774, of T it is -6.307444522537066


In [104]:
T_gs_vec

array([-0.1777779 ,  0.00083138, -0.44008898,  0.00083138, -0.44008898,
        0.00083138, -0.44008898,  0.00083138, -0.44008898,  0.00083138,
       -0.44008898])

In [105]:
S_trans = (V.H @ S[0].to_sparse_matrix @ V).real

In [106]:
T_gs_vec.transpose() * S_trans.toarray() * T_gs_vec

array([[0.01603004, 0.00000014, 0.06753543, 0.00000014, 0.06753543,
        0.00000014, 0.06753543, 0.00000014, 0.06753543, 0.00000014,
        0.06753543],
       [0.00645974, 0.00000069, 0.        , 0.00000069, 0.        ,
        0.00000069, 0.        , 0.00000069, 0.        , 0.00000069,
        0.        ],
       [0.01102062, 0.        , 0.19367831, 0.        , 0.19367831,
        0.        , 0.19367831, 0.        , 0.19367831, 0.        ,
        0.19367831],
       [0.00645974, 0.00000069, 0.        , 0.00000069, 0.        ,
        0.00000069, 0.        , 0.00000069, 0.        , 0.00000069,
        0.        ],
       [0.01102062, 0.        , 0.19367831, 0.        , 0.19367831,
        0.        , 0.19367831, 0.        , 0.19367831, 0.        ,
        0.19367831],
       [0.00645974, 0.00000069, 0.        , 0.00000069, 0.        ,
        0.00000069, 0.        , 0.00000069, 0.        , 0.00000069,
        0.        ],
       [0.01102062, 0.        , 0.19367831, 0.        , 0.

In [113]:
from symmer.projection import QubitTapering
qt = QubitTapering(H)

for w in v_list:
    print(w.sectors_present(S))
    H_tap = qt.taper_it(ref_state=w.state_matrix[0])
    print(exact_gs_energy(H_tap.to_sparse_matrix)[0] - gs_nrg)

[0.5071998 +0.j 0.46118653+0.j 0.13633313+0.j 0.26142689+0.j]
0.2924975159807839
[ 1.+0.j -1.+0.j  1.+0.j  1.+0.j]
0.2924975159807839
[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
0.00992117501148293
[ 1.+0.j -1.+0.j  1.+0.j  1.+0.j]
0.2924975159807839
[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
0.00992117501148293
[ 1.+0.j -1.+0.j  1.+0.j  1.+0.j]
0.2924975159807839
[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
0.00992117501148293
[ 1.+0.j -1.+0.j  1.+0.j  1.+0.j]
0.2924975159807839
[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
0.00992117501148293
[ 1.+0.j -1.+0.j  1.+0.j  1.+0.j]
0.2924975159807839
[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
0.00992117501148293
